In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests, json

In [2]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo'
r = requests.get(url)
data = r.json()

In [3]:
data['Time Series (5min)']

{'2025-05-22 19:55:00': {'1. open': '258.4000',
  '2. high': '258.4000',
  '3. low': '258.4000',
  '4. close': '258.4000',
  '5. volume': '2'},
 '2025-05-22 19:50:00': {'1. open': '258.0600',
  '2. high': '258.4200',
  '3. low': '258.0600',
  '4. close': '258.4200',
  '5. volume': '129'},
 '2025-05-22 19:45:00': {'1. open': '260.8000',
  '2. high': '260.8000',
  '3. low': '260.8000',
  '4. close': '260.8000',
  '5. volume': '1'},
 '2025-05-22 19:40:00': {'1. open': '258.2500',
  '2. high': '258.2500',
  '3. low': '258.0600',
  '4. close': '258.0600',
  '5. volume': '20'},
 '2025-05-22 19:35:00': {'1. open': '259.2600',
  '2. high': '259.2600',
  '3. low': '258.0600',
  '4. close': '258.0600',
  '5. volume': '7'},
 '2025-05-22 19:25:00': {'1. open': '258.3600',
  '2. high': '260.8000',
  '3. low': '258.3600',
  '4. close': '260.8000',
  '5. volume': '2'},
 '2025-05-22 19:20:00': {'1. open': '258.3600',
  '2. high': '258.3600',
  '3. low': '258.3600',
  '4. close': '258.3600',
  '5. volu

In [4]:
data['Meta Data']

{'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
 '2. Symbol': 'IBM',
 '3. Last Refreshed': '2025-05-22 19:55:00',
 '4. Interval': '5min',
 '5. Output Size': 'Compact',
 '6. Time Zone': 'US/Eastern'}

In [5]:
ibm = pd.DataFrame(data['Time Series (5min)']).T

In [6]:
current_date = pd.to_datetime('today').strftime('%Y_%m_%d')

In [7]:
ibm.head()

,1. open,2. high,3. low,4. close,5. volume
2025-05-22 19:55:00,258.4000,258.4000,258.4000,258.4000,2
2025-05-22 19:50:00,258.0600,258.4200,258.0600,258.4200,129
2025-05-22 19:45:00,260.8000,260.8000,260.8000,260.8000,1
2025-05-22 19:40:00,258.2500,258.2500,258.0600,258.0600,20
2025-05-22 19:35:00,259.2600,259.2600,258.0600,258.0600,7
